In [22]:
## Import libraries 
import matplotlib.pyplot as plt
import requests as requests
import pandas as pd
import seaborn as sb
import numpy as np
import csv
from citipy import citipy

In [23]:
##Pull CSV 
source = "./updated_data.csv"

In [26]:
##Assign CSV to dataframe, create duplicate dataframe for future use 
source_df = pd.read_csv(source)
final_df = source_df
source_df.head()

,citeID,full_citation,section
0,6772,"Agrawal, R. et al. (1993). ""Mining Association...",References
1,6773,"Alexander, C. (1964). Notes on the Synthesis o...",References
2,6774,"Arazy, O., Kumar, N., and Shapira, B. (2010). ...",References
3,6775,"Baskerville, R. and V. Vaishnavi. ""A Novel App...",References
4,6776,"Berger, P. and Luckman, T. (1966). The Socia...",References


In [27]:
##Confirm column names 
for col_name in source_df.columns: 
    print(col_name)

citeID
full_citation
section


In [28]:
##Split citation 
source_df = source_df['full_citation'].str.split('(',1, expand=True)
source_df = source_df.rename(columns={0: 'authors', 1: 'year'})
source_df.head()

,authors,year
0,"Agrawal, R. et al.","1993). ""Mining Association Rules between Sets ..."
1,"Alexander, C.",1964). Notes on the Synthesis of Form. Cambrid...
2,"Arazy, O., Kumar, N., and Shapira, B.","2010). ""A Theory-Driven Design Framework for S..."
3,"Baskerville, R. and V. Vaishnavi. ""A Novel App...","J. Brocke, A. Hevner, and A. Maedche, Eds.), S..."
4,"Berger, P. and Luckman, T.",1966). The Social Construction of Reality: A T...


In [31]:
##Create authors df, drop remaining citation
authors_df = source_df
authors_df = authors_df.drop(['year'], axis=1)
authors_df.head()

,authors
0,"Agrawal, R. et al."
1,"Alexander, C."
2,"Arazy, O., Kumar, N., and Shapira, B."
3,"Baskerville, R. and V. Vaishnavi. ""A Novel App..."
4,"Berger, P. and Luckman, T."


In [32]:
##Combine authors and final_df 
final_df = pd.concat([final_df, authors_df], axis=1)
final_df.head()

,citeID,full_citation,section,authors
0,6772,"Agrawal, R. et al. (1993). ""Mining Association...",References,"Agrawal, R. et al."
1,6773,"Alexander, C. (1964). Notes on the Synthesis o...",References,"Alexander, C."
2,6774,"Arazy, O., Kumar, N., and Shapira, B. (2010). ...",References,"Arazy, O., Kumar, N., and Shapira, B."
3,6775,"Baskerville, R. and V. Vaishnavi. ""A Novel App...",References,"Baskerville, R. and V. Vaishnavi. ""A Novel App..."
4,6776,"Berger, P. and Luckman, T. (1966). The Socia...",References,"Berger, P. and Luckman, T."


In [35]:
##Delete Authors column
del source_df['authors']
source_df.head()

,year
0,"1993). ""Mining Association Rules between Sets ..."
1,1964). Notes on the Synthesis of Form. Cambrid...
2,"2010). ""A Theory-Driven Design Framework for S..."
3,"J. Brocke, A. Hevner, and A. Maedche, Eds.), S..."
4,1966). The Social Construction of Reality: A T...


In [36]:
##Strip citation after next ) to capture year 
source_df = source_df['year'].str.split(')',1, expand=True)
source_df = source_df.rename(columns={0: 'year', 1: 'rest'})
source_df.head()

,year,rest
0,1993,". ""Mining Association Rules between Sets of It..."
1,1964,". Notes on the Synthesis of Form. Cambridge, M..."
2,2010,". ""A Theory-Driven Design Framework for Social..."
3,"J. Brocke, A. Hevner, and A. Maedche, Eds.",", Springer, pp. 17-42, 2020."
4,1966,. The Social Construction of Reality: A Treati...


In [37]:
##Drop year from new df 
year_df = source_df
year_df = year_df.drop(['rest'], axis=1 )
year_df.head()

,year
0,1993
1,1964
2,2010
3,"J. Brocke, A. Hevner, and A. Maedche, Eds."
4,1966


In [38]:
##Combine year into final_df 
final_df = pd.concat([final_df, year_df], axis=1)
final_df.head()

,citeID,full_citation,section,authors,year
0,6772,"Agrawal, R. et al. (1993). ""Mining Association...",References,"Agrawal, R. et al.",1993
1,6773,"Alexander, C. (1964). Notes on the Synthesis o...",References,"Alexander, C.",1964
2,6774,"Arazy, O., Kumar, N., and Shapira, B. (2010). ...",References,"Arazy, O., Kumar, N., and Shapira, B.",2010
3,6775,"Baskerville, R. and V. Vaishnavi. ""A Novel App...",References,"Baskerville, R. and V. Vaishnavi. ""A Novel App...","J. Brocke, A. Hevner, and A. Maedche, Eds."
4,6776,"Berger, P. and Luckman, T. (1966). The Socia...",References,"Berger, P. and Luckman, T.",1966


In [39]:
##Remove year
source_df.head()

,rest
0,". ""Mining Association Rules between Sets of It..."
1,". Notes on the Synthesis of Form. Cambridge, M..."
2,". ""A Theory-Driven Design Framework for Social..."
3,", Springer, pp. 17-42, 2020."
4,. The Social Construction of Reality: A Treati...


In [40]:
#Remove period and quote from beginning of string 
source_df['rest'] = source_df['rest'].str.replace('.', '',1)
source_df['rest'] = source_df['rest'].str.replace('"', '',1)
source_df.head()

,rest
0,Mining Association Rules between Sets of Item...
1,"Notes on the Synthesis of Form. Cambridge, MA..."
2,A Theory-Driven Design Framework for Social R...
3,", Springer, pp 17-42, 2020."
4,The Social Construction of Reality: A Treatis...


In [41]:
#Split rest on second " from title 
source_df = source_df['rest'].str.split('"',1, expand=True)
source_df = source_df.rename(columns={0: 'title', 1: 'rest'})
source_df.head()

,title,rest
0,Mining Association Rules between Sets of Item...,Proceedings of the 1993 ACM SIGMOD Conference...
1,"Notes on the Synthesis of Form. Cambridge, MA...",None
2,A Theory-Driven Design Framework for Social R...,Journal of the Association for Information Sy...
3,", Springer, pp 17-42, 2020.",None
4,The Social Construction of Reality: A Treatis...,None


In [42]:
##Add title to final_df 
title_df = source_df
title_df = title_df.drop(['rest'], axis=1 )
final_df = pd.concat([final_df, title_df], axis=1)
final_df.head()

,citeID,full_citation,section,authors,year,title
0,6772,"Agrawal, R. et al. (1993). ""Mining Association...",References,"Agrawal, R. et al.",1993,Mining Association Rules between Sets of Item...
1,6773,"Alexander, C. (1964). Notes on the Synthesis o...",References,"Alexander, C.",1964,"Notes on the Synthesis of Form. Cambridge, MA..."
2,6774,"Arazy, O., Kumar, N., and Shapira, B. (2010). ...",References,"Arazy, O., Kumar, N., and Shapira, B.",2010,A Theory-Driven Design Framework for Social R...
3,6775,"Baskerville, R. and V. Vaishnavi. ""A Novel App...",References,"Baskerville, R. and V. Vaishnavi. ""A Novel App...","J. Brocke, A. Hevner, and A. Maedche, Eds.",", Springer, pp 17-42, 2020."
4,6776,"Berger, P. and Luckman, T. (1966). The Socia...",References,"Berger, P. and Luckman, T.",1966,The Social Construction of Reality: A Treatis...


In [48]:
##Delete title from source df
del source_df['title']
source_df.head()

,rest
0,Proceedings of the 1993 ACM SIGMOD Conference...
1,None
2,Journal of the Association for Information Sy...
3,None
4,None


In [50]:
##Split on next (  to grab publication 
source_df = source_df['rest'].str.split('(',1, expand=True)
source_df.head()

,0,1
0,Proceedings of the 1993 ACM SIGMOD Conference...,None
1,None,None
2,Journal of the Association for Information Sy...,9): 455–490.
3,None,None
4,None,None


In [51]:
##Rename columns 
source_df = source_df.rename(columns={0: 'publication', 1: 'page_numbers'})

In [52]:
##Add rows to final_df 
publication_df = source_df
final_df = pd.concat([final_df, publication_df], axis=1)
final_df.head()

,citeID,full_citation,section,authors,year,title,publication,page_numbers
0,6772,"Agrawal, R. et al. (1993). ""Mining Association...",References,"Agrawal, R. et al.",1993,Mining Association Rules between Sets of Item...,Proceedings of the 1993 ACM SIGMOD Conference...,None
1,6773,"Alexander, C. (1964). Notes on the Synthesis o...",References,"Alexander, C.",1964,"Notes on the Synthesis of Form. Cambridge, MA...",None,None
2,6774,"Arazy, O., Kumar, N., and Shapira, B. (2010). ...",References,"Arazy, O., Kumar, N., and Shapira, B.",2010,A Theory-Driven Design Framework for Social R...,Journal of the Association for Information Sy...,9): 455–490.
3,6775,"Baskerville, R. and V. Vaishnavi. ""A Novel App...",References,"Baskerville, R. and V. Vaishnavi. ""A Novel App...","J. Brocke, A. Hevner, and A. Maedche, Eds.",", Springer, pp 17-42, 2020.",None,None
4,6776,"Berger, P. and Luckman, T. (1966). The Socia...",References,"Berger, P. and Luckman, T.",1966,The Social Construction of Reality: A Treatis...,None,None


In [55]:
##Rearrange columns
final_df = final_df[['citeID', 'authors', 'year', 'title', 'publication', 'page_numbers', 'section', 'full_citation']]

In [59]:
final_df.head()

,citeID,authors,year,title,publication,page_numbers,section,full_citation
0,6772,"Agrawal, R. et al.",1993,Mining Association Rules between Sets of Item...,Proceedings of the 1993 ACM SIGMOD Conference...,None,References,"Agrawal, R. et al. (1993). ""Mining Association..."
1,6773,"Alexander, C.",1964,"Notes on the Synthesis of Form. Cambridge, MA...",None,None,References,"Alexander, C. (1964). Notes on the Synthesis o..."
2,6774,"Arazy, O., Kumar, N., and Shapira, B.",2010,A Theory-Driven Design Framework for Social R...,Journal of the Association for Information Sy...,9): 455–490.,References,"Arazy, O., Kumar, N., and Shapira, B. (2010). ..."
3,6775,"Baskerville, R. and V. Vaishnavi. ""A Novel App...","J. Brocke, A. Hevner, and A. Maedche, Eds.",", Springer, pp 17-42, 2020.",None,None,References,"Baskerville, R. and V. Vaishnavi. ""A Novel App..."
4,6776,"Berger, P. and Luckman, T.",1966,The Social Construction of Reality: A Treatis...,None,None,References,"Berger, P. and Luckman, T. (1966). The Socia..."


In [60]:
##Export final 
final_df.to_csv('testfile.csv', encoding='utf-8')